In [1]:
import pickle
import os

In [2]:
path = "./data/list"

uncleaned_count = 0

movies_dict = {}

for name in os.listdir(path):
    with open(os.path.join(path, name), "rb") as f:
        movies = pickle.load(f)
        uncleaned_count += len(movies)
        for m in movies:
            movies_dict[m["id"]] = m

In [3]:
print(f"Uncleaned count: {uncleaned_count}")
print(f"Duplicates removed count: {len(movies_dict)}")

Uncleaned count: 31467
Duplicates removed count: 17594


In [4]:
# remove movies with no ratings, directors or actors

to_remove = [k for k, v in movies_dict.items() if v["rate"] == "" or len(v["directors"]) == 0 or len(v["casts"]) == 0]

for i in to_remove:
    del movies_dict[i]

print(f"Cleaned count: {len(movies_dict)}")

Cleaned count: 4289


In [5]:
movies_dict["26925317"]

{'directors': ['韩延'],
 'rate': '7.2',
 'cover_x': 2999,
 'star': '35',
 'title': '动物世界',
 'url': 'https://movie.douban.com/subject/26925317/',
 'casts': ['李易峰', '迈克尔·道格拉斯', '周冬雨', '曹炳琨', '王戈'],
 'cover': 'https://img9.doubanio.com/view/photo/s_ratio_poster/public/p2525528688.jpg',
 'id': '26925317',
 'cover_y': 4181}

In [6]:
# 4000 movies to scrape let's go

import aiohttp
from common import headers, proxies
import random
import time

In [14]:
proxy_it = iter(proxies)

def get_next_proxy():
    return f"http://{next(proxy_it)}"

In [15]:
# https://stackoverflow.com/questions/27092833/unicodeencodeerror-charmap-codec-cant-encode-characters

proxy = get_next_proxy()
start_time = time.time()
count = 0

async with aiohttp.ClientSession(headers=headers) as session:
    for i, m in movies_dict.items():
        name = f"./data/detail/{i}.html"
        count += 1
        if os.path.isfile(name):
            continue

        # try until no exception or response is 200
        # yeah the nesting is pretty jank but whatever
        while True:
            try:
                async with session.get(m["url"], proxy=proxy) as resp:
                    if resp.status == 200:
                        result = await resp.text()
                        if not result.startswith("<script>"):
                            with open(name, 'w', encoding="utf-8") as f:
                                f.write(result)
                            break
                    proxy = get_next_proxy()
                    print(f"Response not 200, switching proxy to {proxy}")
            except Exception as e:
                proxy = get_next_proxy()
                print(f"{e}, switching proxy to {proxy}")

        print(f"Scraped {m['title']}, id {i}, time elapsed {time.time() - start_time}, {len(movies_dict) - count} to go")
        time.sleep(random.random())

[WinError 10054] An existing connection was forcibly closed by the remote host, switching proxy to http://106.75.86.177:80
400, message='Bad Request', url=URL('http://106.75.86.177:80'), switching proxy to http://121.226.248.241:8118
Cannot connect to host 121.226.248.241:8118 ssl:default [Connect call failed ('121.226.248.241', 8118)], switching proxy to http://219.138.229.131:9091
400, message='Bad Request', url=URL('http://219.138.229.131:9091'), switching proxy to http://120.26.14.114:8888
Response not 200, switching proxy to http://121.37.227.255:8888
Scraped 止杀令, id 21357362, time elapsed 37.08364200592041, 3144 to go
Scraped 旱地忽律朱贵, id 20432985, time elapsed 39.92454195022583, 3143 to go
Scraped 爱情银行, id 11632586, time elapsed 43.007245779037476, 3142 to go
Scraped 囧人之越挠越痒, id 10545947, time elapsed 46.194018840789795, 3141 to go
Scraped 国徽, id 19965216, time elapsed 49.462395429611206, 3140 to go
Scraped 红财神, id 21359495, time elapsed 52.9320228099823, 3139 to go
Scraped 阆中之恋, 

RuntimeError: coroutine raised StopIteration

In [15]:
# check number of scraped files is correct
_, _, files = next(os.walk("./data/detail"))
len(files)

1187